#### Name: Qiwen Ma

Date: 5/4/2018

Version: 3.0

Environment: Python 3.6.0 and Jupyter notebook

Libraries used:
* json (for json file, included in Anaconda Python 3.6) 
* re 2.2.1 (for regular expression, included in Anaconda Python 3.6) 

## 1.Introduction
This assignment make extraction of the data from an xml file and save the result into a csv file. The requirement can be explain as follow:
1. The data structure is json. All the data should be save in a json file.
2. Use only the regular expression to extract data.

The detail of each section is shown below.

In [ ]:
import re
import json

To open the xml file and use readlines method to present the data.

In [ ]:
xml = open('australian-sport-thesaurus-student.xml', encoding= 'UTF-8')
xml_lines = xml.readlines()
#xml_lines

Use the regular expression to find out the data in exact tag of the xml file.

In [ ]:
reg_description = re.compile('(?<=<Description>)(.*?)(?=</Description>)')
reg_start = re.compile('<Description>')
reg_description_end = re.compile('</Term>')
reg_related_terms = re.compile('<RelatedTerms>')
reg_related_title = re.compile('(?<=<Title>)(.*?)(?=</Title>)')
reg_title_end = re.compile('</RelatedTerms>')
reg_relationship = re.compile('(?<=<Relationship>)(.*?)(?=</Relationship>)')

After observation, the tag 'description','title' and 'related terms' are in one-level list. To find out the data inside these three tags and append in to a new list.

In [ ]:
dic = {}
dic["thesaurus"] = []
for i in range(0,len(xml_lines)):
    dic1 = {}
    c = reg_description.search(xml_lines[i])
    #to find out the lines with 'description' tag and locate them
    if reg_start.search(xml_lines[i]) != None:
        #use the 'description' location to search for it previous and next tag and values
        d = reg_related_title.search(xml_lines[i-1])
        term = reg_description_end.search(xml_lines[i+1])
        #if the data only have 'description' and 'title', append to a dictionary
        if c != None and d != None:
            if term != None:
                dic1 = {"Description": c.group(), "Title": d.group()}
            #if not, keep a null lits as the value of the 'related terms'
            else:
                dic1 = {"Description": c.group(),"RelatedTerms": [], "Title": d.group()}
            dic["thesaurus"].append(dic1)
#print(dic)

Then to structure the sublist of the 'related terms'. The data of the in the 'related term' tag is with tag 'title' and 'relationship'. But it can contain more than one pair of them in one 'related term'.

In [ ]:
second_list = []
dic3 = {}
for k in range(0,len(xml_lines)):
    dic_related = {}
    dic_related["RelatedTerms"] = []
    #to locate the 'related terms' tag and find out it sub-tags
    if reg_related_terms.search(xml_lines[k]) != None:
        k += 2
        #find out the 'title' and 'relationship' content of inside 'related terms' tag
        a = reg_related_title.search(xml_lines[k])
        relationship = reg_relationship.search(xml_lines[k + 1])
        if a != None and relationship != None:
            #structure the sub-dictionary of 'related terms' directly,
            #if there is only one pair of 'title' and 'relationship'
            dic3 = {"Relationship": relationship.group(),"Title": a.group()}
            dic_related["RelatedTerms"].append(dic3)
            k += 3
            #if the 'title' and 'relationship' tag pair is more than one,
            #find out all of them and structure them in sub-dictionary
            while reg_title_end.search(xml_lines[k]) == None:
                k += 1
                b = reg_related_title.search(xml_lines[k])
                relationship = reg_relationship.search(xml_lines[k + 1])
                if b != None and relationship != None:
                    dic3 = {"Relationship": relationship.group(), "Title": b.group()}
                    dic_related["RelatedTerms"].append(dic3)
        #print(dic_related)
        second_list.append(dic_related)

Afer extracting the data in one-level dictionary and sub-dictionary, to assign the 'related terms' sub-dictionary into the original null list. The aggregation of the two dictionaries makes the whole structure finished.

In [ ]:
reg = re.compile('RelatedTerms')
s= 0
for dics in dic["thesaurus"]:
    if reg.search(str(dics)) !=None:        
        dics["RelatedTerms"] = second_list[s]["RelatedTerms"]
        s += 1
dics["RelatedTerms"] = second_list[s]["RelatedTerms"]

In [ ]:
dic

Finally, to dump the data in to using json and save the data in to json file.

In [ ]:
data = json.dumps(dic,sort_keys = True)
data

In [ ]:
with open('sport.dat.json','w') as file:
    json.dump(dic,file,sort_keys = True)

## 3. Summary

After finishing this assignment task, what I have learned is concluded as follow:
1. Using the regular expression to locate the content and find out the content of each lines.
2. The xml file contain different situation of structure. It is important to observe the data, especially the with more than one 'title' and 'relationship' tag pairs in one 'related terms' tag, and there may be no 'related terms' tag in one 'title' and 'description' tag pair.